In [ ]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
solver = "highs"

In this exercise, we will working with `link` component in PyPSA and go through different applications of links. The exercise consists of three main checkpoints:
- Simulating energy flow between two buses with link
- Simulating energy carrier conversion in PyPSA
- Simulating infrastructure with multiple outputs using customised link

*** 
Check-point 1:

**Simulating energy flow between two buses with link**

*Build a network in PyPSA with the two buses North and South and attach the `100MW` electricity load at each bus. North bus is attached with a `gas_power_plant`, while South is attached by a `solar farm` with flexible capacities. All plants' techno-economic parameters are provided. Afterward, use `link` to connect the two buses and observe the power exchange between two bus.* 

<img src="../data/block diagrams/d1_e3-1.svg" width=500/>

Initialize network

In [ ]:
# Create empty PyPSA network
network = pypsa.Network()
# Set snapshots to the year 2023 and at hourly resolution
snapshots = pd.date_range("01-01-2023", "01-01-2024", freq="h", inclusive="left")
network.set_snapshots(snapshots)

Add bus `North` and `South` to the network

In [ ]:
???

Add `200MW` load to each bus

In [ ]:
# You can 'madd' instead of 'add' to add multiple components at once
???

Add power plants to each bus

In [ ]:
# We need to calculate annualized capital expenditure
def calculate_annualised_capex(capex: float, interest: float, lifetime: int):
    crf = (
        interest * (1 + interest) ** lifetime / ((1 + interest) ** lifetime - 1)
    )  # Capital recovery factor
    return capex * crf

In [ ]:
# Techno-economic parameters of the gas-fired combined cycle power plant (CCGT):
lifetime = 25  # years
interest = 0.05  # unit: -
CAPEX = 904779  # $/MW
FOM = 0.0334 * CAPEX  # $/MW per year (3.34% of CAPEX)
VOM = 4.55  # $/MWh variable
fuel_cost = 24.57  # $/MWh_th per unit gas consumed
efficiency = 0.57

In [ ]:
???

In [ ]:
# Import an example daily pattern
solar_pattern = pd.read_csv("../data/weather data/example_solar_daily_pattern.csv")[
    "daily pattern"
]
# annual time-series availability of solar farm (just a simplified example)
yearly_avail = pd.Series(list(solar_pattern) * 365, index=network.snapshots)

In [ ]:
# Techno-economic parameters of the solar farm:
lifetime = 37.5  # years
interest = 0.05  # unit: -
CAPEX = 676570  # $/MW
FOM = 0.017275 * CAPEX  # $/MW per year (1.7275% of CAPEX)
VOM = 0.0106  # $/MWh
fuel_cost = 0  # per unit of solar irradiation consumed

In [ ]:
???

Add a bi-directional link between `North` and `South`

In [ ]:
network.add(class_name="Link", 
            name="north_to_south_transmission",
            bus0=???,
            bus1=???,
            p_nom_extendable=???, # allow flexible capacity expansion
            p_min_pu=???) # to allow bi-directional flow

Now try to solve the network and check the results

In [ ]:
# Solve network using cbc solver
network.optimize(solver_name=solver)

In [ ]:
# inspect transmission capacity
???

In [ ]:
# inspect power plants' capacities
???

In [ ]:
# Inspect transmission flow interaction with other power plants to supply loads at north bus
???

Exporting check-point 1 network

In [ ]:
network.export_to_netcdf("../results/network_d1_e3-1.nc")

*** 
Check-point 2:

**Simulating energy carrier conversion in PyPSA**

*In this exercise, we will simulate a gas-fired power plant using the `link` component instead of the `generator`. To set it up, we will create a theoretical gas supply bus (With theoretical gas supply `generator` attached to it) and connect it to an electricity bus with an electrical load through the `link` component* 

<img src="../data/block diagrams/d1_e3-2.svg" width=200/>

Initialize network

In [ ]:
# Create empty PyPSA network
network = pypsa.Network()
# Set snapshots to the year 2023 and at hourly resolution
snapshots = pd.date_range("01-01-2023", "01-01-2024", freq="h", inclusive="left")
network.set_snapshots(snapshots)
network.snapshots

Add electricity bus and `100MW` load 

In [ ]:
# add electricity bus
???
# add electricity load
???

Add theoretical gas bus with a theroretical gas supply `generator` 

In [ ]:
# Techno-economic parameters of the gas-fired combined cycle power plant (CCGT):
lifetime = 25  # years
interest = 0.05  # unit: -
CAPEX = 904779  # $/MW
FOM = 0.0334 * CAPEX  # $/MW per year (3.34% of CAPEX)
VOM = 4.55  # $/MWh variable
fuel_cost = 24.57  # $/MWh_th per unit gas consumed
efficiency = 0.57

In [ ]:
# add gas bus
network.add(class_name="Bus", name="gas_hub", carrier='gas')
# add theoretical gas supply generator
network.add(class_name="Generator",
  name=???,
  bus=???,
  carrier=???,
  marginal_cost=???, # Cost per thermal unit gas consumed
  p_nom_extendable=???, # assuming unlimited supply
  )

Add link representing gas-fired power plant converting gas into electricity.

> Note: The marginal and capital costs in `links` are defined for the inlet bus (`bus0`), before efficiency losses. Therefore, the marginal and capital costs for the gas-fired power plant which are given per unit of produced electricity, should be converted into the costs per unit of supplied gas by multiplying to the efficiency.

In [ ]:
annualized_capex = calculate_annualised_capex(capex=CAPEX, interest=interest, lifetime=lifetime)

network.add(class_name="Link",
      name="gas_power_plant",
      bus0="gas_hub",
      bus1="electricity",
      marginal_cost= ???, # All cost input attributes have to be converted from elec to thermal by multiplying efficiency.
      capital_cost= ???,  # All cost input attributes have to be converted from elec to thermal by multiplying efficiency
      p_nom = ??? # All capacity input attributes have to be converted from elec to thermal by dividing efficiency
      p_nom_extendable=True,
      efficiency=efficiency,
      )

Now try to solve the network and check the results

In [ ]:
# Solve network using cbc solver
network.optimize(solver_name=solver)

In [ ]:
# inspect gas plant capacity
???

Exporting check-point 2 network

In [ ]:
network.export_to_netcdf("../results/network_d1_e3-2.nc")

*** 
Check-point 3:

**Simulating infrastructure with multiple outputs using customised link**

*A `link` component in PyPSA can be configured to support multiple outputs. In this exercise, we will modify the gas_power_plant from the previous task to simulate and account for its CO2 emission releasing to the atmosphere as a secondary output alongside electricity generation.*

> **Hint:** Links can also be defined with multiple outputs in fixed ratio to the power in the single input by defining new columns bus2, bus3, etc. in network.links along with associated columns for efficiency2, efficiency3, etc. The different outputs are then equal to the input multiplied by the corresponding efficiency. For more information about this, checkout [PyPSA documentation about multi-link](https://pypsa.readthedocs.io/en/latest/user-guide/components.html#multilink)

<img src="../data/block diagrams/d1_e3-3.svg" width=500/>

Import checkpoint point 2 network

In [ ]:
network = pypsa.Network("../results/network_d1_e3-2.nc")

We first need to create an atmosphere bus with a CO2 theoretical storage. When you need a component to store only without discharge, PyPSA `Store` is recommended.

In [ ]:
# adding an atmostphere bus
???

# adding theoretical storage to store co2 released to the atmosphere
network.add(
    class_name="Store",
    name="co2_atmosphere_storage",
    e_nom_extendable=True,  # allow infinite co2
    carrier="co2",
    bus="co2_atmosphere",
)

Now we can modify current `gas_power_plant` link to connect it to `co2_atmosphere` with emission factor of `0.2 tCO2/MWhth` 

In [ ]:
# add co2_atmosphere as bus2 of gas link
???
# add emission factor of gas power plant as efficiency2
???

Now try to solve the network and check the results

In [ ]:
# Solve network using cbc solver
network.optimize(solver_name=solver)

In [ ]:
# how much electricity is produced (transfered to bus1=electricity)?
???

how much CO2 is emitted into the atmosphere (transferred to bus2=co2_atmosphere)

In [ ]:
# Option 1: Using output as energy flow going into bus2
???

In [ ]:
# Option 2: Calculating by output of energy flow going out from bus0 * emission factor as efficiency2
???

In [ ]:
# Option 3: Look at the volume of co2 atmosphere storage
???

*** 
### Extra:

**Can you try to model power plant with Carbon capture technology (CCS)?**

> **Hint:** You can considered the captured emission as another byproduct of electricity production, with a fixed ratio relative to the amount released into the atmosphere.